In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import pylab 
import scipy.stats as stats
import statsmodels.api as sm

from numpy import mean
from numpy import median
from numpy import percentile

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import StandardScaler

from pandas import read_csv
from pandas import datetime
from pandas import DataFrame

from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

# 1 Data

## 1.1 Import energy consumption data from CCEE

In [ ]:
df = pd.read_csv('/kaggle/input/ccee-brazil-energy-consumption/CCEE_BR_Data_DEC_2020_v3.csv')


## 1.2 Visualize data

In [ ]:
df.info()
print('Size of df data (rows, columns) = ', df.shape)

## 1.3 Check missing data

In [ ]:
plt.figure(figsize=(14,5))
sns.heatmap(df.isnull(), vmin=0, vmax=1,)

total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

## 1.4 Data organization

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Data'], format='%m/%d/%Y')
df['Ano'] = df['Timestamp'].dt.year
df['Mês'] = df['Timestamp'].dt.month
df['Dia'] = df['Timestamp'].dt.day
df['DiaSem'] = df['Timestamp'].dt.weekday

In [ ]:
df.head(3)

In [ ]:
d=df.groupby(['Mês'])['Consumo']
d.describe()          

In [ ]:
df['Consumo'].describe()

## 1.5 Data analysis

In [ ]:
df['CovidStatus']=df['Covid']
df['CovidStatus'] = df['CovidStatus'].replace(0, 'Pre Covid')
df['CovidStatus'] = df['CovidStatus'].replace(1, 'Covid Crisis')                     

In [ ]:
# exclude 'Exportador'
df=df[df['Classe']!='Exportador']
# ACR = Mercado de Contratação Regulada
ACR_df = df[df['Classe']=='Distribuidor']
# ACL = Mercado de Contratação Livre
ACL_df = df[df['Classe']!='Distribuidor']

# df pré COVID
df0 = df[df['Covid']==0]
# df durante COVID
df1 = df[df['Covid']==1]

# ACL pré COVID
Covid_0_ACL = ACL_df[ACL_df['Covid']==0]
# ACL durante COVID
Covid_1_ACL = ACL_df[ACL_df['Covid']==1]

# ACL pré COVID
Covid_0_ACR = ACR_df[ACR_df['Covid']==0]
# ACL durante COVID
Covid_1_ACR = ACR_df[ACR_df['Covid']==1]

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(16,6))
sns.countplot(x='Classe', hue='Submercado', data=ACL_df,palette='RdBu_r')
plt.title('Distribuição dos consumidores por submercado - ACL', fontsize=20);
plt.xlabel("Classe")
plt.legend(loc=2)
plt.ylabel('Quantidade');

plt.figure(figsize=(16,6))
sns.countplot(hue='Ramo', x='Classe', data=ACL_df,palette='RdBu_r')
plt.title('Distribuição dos consumidores por ramo - ACL', fontsize=20);
plt.xlabel("Classe")
plt.ylabel('Quantidade');

In [ ]:
a=ACL_df.groupby(['Timestamp','Ramo'])['Consumo'].sum()
a=a.groupby(['Ramo']).mean()
a=a.reset_index().sort_values(['Consumo'],ascending=False)

plt.figure(figsize=(16,6))
sns.barplot(x='Ramo', y='Consumo', data=a, palette='RdBu_r');
plt.title('Consumo por ramo - ACL', fontsize=20);
plt.xlabel("Ramo")
plt.xticks(rotation = 45, ha='right');
plt.ylabel('Consumo - MWm');

In [ ]:
a=df.groupby(['Timestamp','Classe'])['Consumo'].sum()
a=a.groupby(['Classe']).mean()
a=a.reset_index().sort_values(['Consumo'],ascending=False)

plt.figure(figsize=(16,6))
sns.barplot(x='Classe', y='Consumo', data=a, palette='viridis');
plt.title('Consumo por Classe - SIN', fontsize=20);
plt.xlabel("Classe")
plt.xticks(rotation = 45, ha='right');
plt.ylabel('Consumo - MWm');

print('Média Consumo Comercializador Varejista = %.2f MWm' % df[df['Classe']=='Comercializador Varejista']['Consumo'].values.mean())
print('Média Consumo Consumidor Especial = %.2f MWm' % df[df['Classe']=='Consumidor Especial']['Consumo'].values.mean())
print('Média Consumo Autoprodutor = %.2f MWm' % df[df['Classe']=='Autoprodutor']['Consumo'].values.mean())
print('Média Consumo Consumidor Livre = %.2f MWm' % df[df['Classe']=='Consumidor Livre']['Consumo'].values.mean())
print('Média Consumo Distribuidor = %.2f MWm' % df[df['Classe']=='Distribuidor']['Consumo'].values.mean())

In [ ]:
a=ACL_df.groupby(['Timestamp','UF'])['Consumo'].sum()
a=a.groupby(['UF']).mean()
a=a.reset_index().sort_values(['Consumo'],ascending=False)

plt.figure(figsize=(16,6))
sns.barplot(x='UF', y='Consumo', data=a, palette='RdBu_r');
plt.title('Consumo por ramo - ACL', fontsize=20);
plt.xlabel("Ramo")
plt.xticks(rotation = 45, ha='right');
plt.ylabel('Consumo - MWm');

In [ ]:
plt.figure(figsize=(18,6))
df0_ts=df0.groupby(['Timestamp'])['Consumo'].sum()/1000
df1_ts=df1.groupby(['Timestamp'])['Consumo'].sum()/1000
df_ts=df.groupby(['Timestamp'])['Consumo'].sum()/1000

plt.title('Consumo Sistema Interligado Nacional', fontsize=20);
plt.ylabel('GWm')
plt.plot(df0_ts, label='Mean pre Covid')
plt.plot(df1_ts, label='Mean during Covid')
plt.plot(df_ts.rolling(window=7,center=False).mean(),label='Rolling Mean 7 days');
plt.grid(b=None, which='major', axis='both')
plt.legend();

In [ ]:
a=ACL_df.groupby(['Ano','DiaSem'])['Consumo'].mean()
a=a.reset_index().sort_values(['Ano'],ascending=True)

plt.figure(figsize=(18,6))
sns.lineplot(x='DiaSem', y='Consumo', data=a, hue='Ano')
plt.title('Consumo - ACL', fontsize=20);
plt.ylabel('Dia Semana')
plt.legend();

a=ACR_df.groupby(['Ano','DiaSem'])['Consumo'].mean()
a=a.reset_index().sort_values(['Ano'],ascending=True)

plt.figure(figsize=(18,6))
sns.lineplot(x='DiaSem', y='Consumo', data=a, hue='Ano')
plt.title('Consumo - ACR', fontsize=20);
plt.ylabel('Dia Semana')
plt.legend();

In [ ]:
pivot_df = ACL_df.pivot_table(values='Consumo',index='DiaSem',columns='Ano')
plt.figure(figsize=(8,6))
plt.title('Consumo - ACL', fontsize=20);
plt.ylabel('Dia Semana')
sns.heatmap(pivot_df.astype(int), annot=True, fmt='g');

pivot_df = ACR_df.pivot_table(values='Consumo',index='DiaSem',columns='Ano')
plt.figure(figsize=(8,6))
plt.title('Consumo - ACR', fontsize=20);
plt.ylabel('Dia Semana')
sns.heatmap(pivot_df.astype(int), annot=True, fmt='g');

In [ ]:
a=ACL_df.groupby(['CovidStatus','DiaSem'])['Consumo'].mean()
a=a.reset_index().sort_values(['CovidStatus'],ascending=False)

plt.figure(figsize=(18,6))
sns.barplot(x='DiaSem', y='Consumo', data=a, hue='CovidStatus')
plt.title('Consumo - ACL', fontsize=20);
plt.ylabel('Dia Semana')
plt.legend( loc='upper center', prop={'size': 12})

a=ACR_df.groupby(['CovidStatus','DiaSem'])['Consumo'].mean()
a=a.reset_index().sort_values(['CovidStatus'],ascending=False)

plt.figure(figsize=(18,6))
sns.barplot(x='DiaSem', y='Consumo', data=a, hue='CovidStatus')
plt.title('Consumo - ACR', fontsize=20);
plt.ylabel('Dia Semana')
plt.legend( loc='upper center', prop={'size': 12})


In [ ]:
pivot_df = ACL_df.pivot_table(values='Consumo',columns='Covid',index='DiaSem')
plt.plot(pivot_df);

pivot_df = ACL_df.pivot_table(values='Consumo',columns='DiaSem',index='Covid')
plt.figure(figsize=(12,2))
plt.title('Consumo Pré / Pós Covid- ACL', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade');
sns.heatmap(pivot_df.astype(int), annot=True, fmt='g');


In [ ]:
pivot_df = ACR_df.pivot_table(values='Consumo',columns='Covid',index='DiaSem')
plt.plot(pivot_df);

pivot_df = ACR_df.pivot_table(values='Consumo',columns='DiaSem',index='Covid')
plt.figure(figsize=(12,2))
plt.title('Consumo Pré / Pós Covid - ACR', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade')
sns.heatmap(pivot_df.astype(int), annot=True, fmt='g');



In [ ]:
ACR_std=ACR_df.groupby(['Timestamp'])['Consumo'].tail(200)
from statistics import stdev 

In [ ]:
ACL_df['CatCons']=pd.qcut(ACL_df['Consumo'], q=5, labels=['A','B','C','D','E'])
ACL_df['CatConsIntervalo']=pd.qcut(ACL_df['Consumo'], q=5)

ACR_df['CatCons']=pd.qcut(ACR_df['Consumo'], q=5, labels=['A','B','C','D','E'])
ACR_df['CatConsIntervalo']=pd.qcut(ACR_df['Consumo'], q=5)

In [ ]:
ACL_df.head(3)

In [ ]:
ACR_df.head(3)

In [ ]:
plt.figure(figsize=(16,6))
sns.countplot(x='Ramo', hue='CatCons', data=ACL_df,palette='viridis')
plt.title('Distribuição dos consumidores', fontsize=20);
plt.xlabel("Classe")
plt.ylabel('Quantidade');
plt.xticks(rotation = 45, ha='right');

plt.figure(figsize=(16,6))
sns.countplot(x='Classe', hue='CatCons', data=ACL_df,palette='viridis')
plt.title('Distribuição dos consumidores', fontsize=20);
plt.xlabel("Classe")
plt.ylabel('Quantidade');
#plt.xticks(rotation = 45, ha='right');


In [ ]:
MC0=mean(Covid_0_ACR['Consumo'].values)
MC1=mean(Covid_1_ACR['Consumo'].values)

In [ ]:
pivot_df = ACL_df.pivot_table(values='Consumo',index='Submercado', columns='CovidStatus')
plt.figure(figsize=(12,3))
plt.title('Consumo Pre/Pos Covid por Submercado', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');

plt.plot(pivot_df.transpose(),'o', markersize=15);
#plt.hist(pivot_df,orientation='horizontal')

In [ ]:
pivot_df = ACL_df.pivot_table(values='Consumo',index='Classe', columns='CovidStatus')
plt.figure(figsize=(12,3))
plt.title('Consumo Pre/Pos Covid por Classe', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');

In [ ]:
pivot_df = ACL_df.pivot_table(values='Consumo',index='UF', columns='CovidStatus')
plt.figure(figsize=(16,2))
plt.title('Consumo Pre/Pos Covid por UF', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');

In [ ]:
ACR_df['UF'].unique()

In [ ]:
ACL_df['UF'].unique()

In [ ]:
df['UF'].unique()

In [ ]:
pd.options.display.float_format = "{:.2f}".format

pivotC0_df = Covid_0_ACR.pivot_table(values='Consumo',columns='UF')
pivotC1_df = Covid_1_ACR.pivot_table(values='Consumo',columns='UF')
pivotC_df = (1-pivotC1_df/pivotC0_df.values.astype(int))*100
plt.figure(figsize=(18,1))
plt.title('Queda de Consumo (%)', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade')
sns.heatmap(pivotC_df, annot=True, cmap='Oranges');

plt.figure(figsize=(18,6));
sns.barplot( data=pivotC_df, color='black');

In [ ]:
pd.options.display.float_format = "{:.2f}".format
pivotC0_df = Covid_0_ACL.pivot_table(values='Consumo',columns='Ramo')
pivotC1_df = Covid_1_ACL.pivot_table(values='Consumo',columns='Ramo')
pivotC_df = (1-pivotC1_df/pivotC0_df.values.astype(int))*100
plt.figure(figsize=(18,1))
sns.heatmap(pivotC_df, annot=True, cmap='Oranges')
plt.title('Queda de Consumo (%)', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('Quantidade')
plt.xticks(rotation = 45, ha='right');

plt.figure(figsize=(18,6))
sns.barplot( data=pivotC_df)
plt.xticks(rotation = 45, ha='right');


In [ ]:
plt.figure(figsize=(16,4))
plt.plot(pivotC1_df.transpose(),'o', label='Crise Covid',markersize=15)
plt.plot(pivotC0_df.transpose(),'o', label='2018-2019',markersize=15)
#plt.hist(pivot_df,orientation='horizontal')
plt.title('Consumo de energia elétrica por ramo de atividade (MWm)', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Consumo (MWm)')
plt.legend()
plt.grid(b=None, which='major', axis='both')
plt.ylim(0,140)
plt.xticks(rotation = 45, ha='right');

In [ ]:
pd.options.display.float_format = "{:.2f}".format
pivotC0_df = Covid_0_ACL.pivot_table(values='Consumo',columns='Ramo')
pivotC1_df = Covid_1_ACL.pivot_table(values='Consumo',columns='Ramo')
pivotC_df = (1-pivotC1_df/pivotC0_df.values.astype(int))*100
plt.figure(figsize=(20,1))
sns.heatmap(pivotC_df, annot=True, cmap='Oranges')
plt.title('Queda de Consumo (%)', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('')
plt.xticks(rotation = 45, ha='right');

plt.figure(figsize=(16,3))
sns.barplot(data=pivotC_df)
plt.grid(b=None, which='major', axis='both')
plt.xticks(rotation = 45, ha='right');

# colocar os números nos markers!!!


In [ ]:
xxx = pivotC_df.values.flatten()
yyy = pivotC1_df.values.flatten()
zzz = pivotC0_df.values.flatten()

kkk=pd.Series(xxx.transpose())
lll=pd.Series(yyy.transpose())
mmm=pd.Series(zzz.transpose())

cons=ACL_df.groupby(['Ramo'])['Consumo'].mean()
list_ramo=ACL_df['Ramo'].unique()


In [ ]:
#total = df.isnull().sum().sort_values(ascending = False)
#percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)

scatter1  = pd.concat([kkk,mmm,lll], axis=1, keys=['Queda de consumo (%)', 'Consumo_Covid_0','Consumo_Covid_1'])
scatter1.head(6)

scatter1['Ramo de atividade'] = list_ramo


In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(x='Queda de consumo (%)', y='Consumo_Covid_0', data=scatter1, color='grey', s=1000, alpha=0.5)
sns.scatterplot(x='Queda de consumo (%)', y='Consumo_Covid_1', hue='Ramo de atividade', data=scatter1, palette='twilight_shifted_r', s=1000)
plt.legend(loc='best', fontsize='medium')
plt.grid(b=None, which='major', axis='both')
plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#plt.text(10, 80,"WEF@Davos", color="gray")

plt.ylabel('Consumo (MWm)');


#plt.figure(figsize=(10,10))
#plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#sns.scatterplot(x='Consumo_Covid_0', y='Consumo_Covid_1', hue='Ramo de atividade', data=scatter1, color='grey', s=1000, alpha=0.5)
##sns.scatterplot(x='Queda de consumo (%)', y='Consumo_Covid_1', hue='Ramo de atividade', data=scatter1, palette='twilight_shifted_r', s=1000)
#plt.legend(loc='upper left', fontsize='small')
#plt.grid(b=None, which='major', axis='both')
#plt.ylabel('Consumo (MWm)')
#plt.axis(aspect='equal')
#plt.xlim(0,80)
#plt.ylim(0,80);
#plt.text(2020.01, 412.4-0.5,"WEF@Davos", color="gray")

In [ ]:
scatter1  = pd.concat([kkk,mmm,lll], axis=1, keys=['Queda de consumo (%)', 'Consumo_Covid_0','Consumo_Covid_1'])
scatter1.head(6)

scatter1['Ramo de atividade'] = list_ramo

ts=ACL_df.groupby(['Timestamp','Ramo','CovidStatus'])['Consumo'].mean()

ts_df = pd.DataFrame(ts)

ts_df

ts_df = ts_df.reset_index()

In [ ]:
plt.figure(figsize=(16,6))
sns.barplot(x='Ramo', y='Consumo', data=ts_df, hue='CovidStatus')
plt.title('Consumo de energia elétrica por ramo de atividade (MWm)', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Consumo (MWm)')
#plt.legend()
plt.grid(b=None, which='major', axis='both')
plt.ylim(0,200)
plt.xticks(rotation = 45, ha='right');
#a.legend(title='Legenda', loc='upper right');

ts=df.groupby(['Submercado','CovidStatus'])['Consumo'].mean()
ts=ts.reset_index()

In [ ]:
ts_df.head(3)

In [ ]:
plt.figure(figsize=(16,6))
a=sns.barplot(x='Ramo', y='Consumo', data=ts_df, hue='CovidStatus')
plt.title('Consumo de energia elétrica por ramo de atividade (MWm)', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Consumo (MWm)')
#plt.legend()
plt.grid(b=None, which='major', axis='both')
plt.ylim(0,180)
a.legend(title='Legenda', loc='upper left');
plt.xticks(rotation = 45, ha='right');


In [ ]:
ts=df[df['Classe']!='Distribuidor']
ts=ts.groupby(['UF','CovidStatus'])['Consumo'].mean()
ts=ts.reset_index()
plt.figure(figsize=(16,6))
a=sns.barplot(x='UF', y='Consumo', data=ts, hue='CovidStatus')
plt.title('Consumo de energia elétrica por UF (MWm)', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Consumo (MWm)')
#plt.legend()
plt.grid(b=None, which='major', axis='both')
plt.ylim(0,200)
a.legend(title='Legenda', loc='upper left');
#plt.xticks(rotation = 0, ha='right');

In [ ]:
ts=df[df['Classe']!='Distribuidor']
ts=ts.groupby(['Classe','CovidStatus'])['Consumo'].mean()
ts=ts.reset_index()
plt.figure(figsize=(16,6))
a=sns.barplot(x='Classe', y='Consumo', data=ts, hue='CovidStatus')
plt.title('Consumo de energia elétrica por Classe (MWm)', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Consumo (MWm)')
#plt.legend()
plt.grid(b=None, which='major', axis='both')
plt.ylim(0,80)
a.legend(title='Legenda', loc='upper left');
#plt.xticks(rotation = 0, ha='right');


In [ ]:
pd.options.display.float_format = "{:.2f}".format
pivotC_S0_df = Covid_0_ACL.pivot_table(values='Consumo',columns='Submercado')
pivotC_S1_df = Covid_1_ACL.pivot_table(values='Consumo',columns='Submercado')
pivotC_df = (1-pivotC_S1_df/pivotC_S0_df.values.astype(int))*100
plt.figure(figsize=(8,1))
sns.heatmap(pivotC_df, annot=True, cmap='Oranges')
plt.title('Queda de Consumo por Submercado (%)', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('')
plt.xticks(rotation = 45, ha='right');




In [ ]:
xxx = pivotC_df.values.flatten()
yyy = pivotC_S1_df.values.flatten()
zzz = pivotC_S0_df.values.flatten()

kkk=pd.Series(xxx.transpose())
lll=pd.Series(yyy.transpose())
mmm=pd.Series(zzz.transpose())

In [ ]:
cons=ACL_df.groupby(['Submercado'])['Consumo'].mean()

pivotC_df.values


list_submercado=ACL_df['Submercado'].unique()
list_submercado

In [ ]:
scatter1  = pd.concat([kkk,mmm,lll], axis=1, keys=['Queda de consumo (%)', 'Consumo_Covid_0','Consumo_Covid_1'])
scatter1.head(6)
scatter1['Submercado'] = list_submercado
scatter_SB=scatter1

In [ ]:
scatter1.head(6)


In [ ]:
pd.options.display.float_format = "{:.2f}".format
pivotC_C0_df = Covid_0_ACL.pivot_table(values='Consumo',columns='Classe')
pivotC_C1_df = Covid_1_ACL.pivot_table(values='Consumo',columns='Classe')
pivotC_df = (1-pivotC_C1_df/pivotC_C0_df.values.astype(int))*100
plt.figure(figsize=(20,1))
sns.heatmap(pivotC_df, annot=True, cmap='Oranges')
plt.title('Queda de Consumo por Classe(%)', fontsize=20);
plt.xlabel("Ramo de atividade")
plt.ylabel('')
plt.xticks(rotation = 45, ha='right');


# Covid data input

In [ ]:
covid_BR_df = pd.read_csv('/kaggle/input/corona-virus-brazil/brazil_covid19_macro.csv')
covid_UF_df = pd.read_csv('/kaggle/input/corona-virus-brazil/brazil_covid19.csv')
#BR_pop = pd.read_csv('kaggle/input/corona-virus-brazil/brazil_population_2019.csv')
BRpop_df = pd.read_csv('/kaggle/input/ccee-brazil-energy-consumption/UF_pop.csv')

In [ ]:
covid_BR_df['Timestamp'] = pd.to_datetime(covid_BR_df['date'], format='%Y/%m/%d')
covid_BR_df['Ano'] = covid_BR_df['Timestamp'].dt.year
covid_BR_df['Mês'] = covid_BR_df['Timestamp'].dt.month
covid_BR_df['Dia'] = covid_BR_df['Timestamp'].dt.day
covid_BR_df['DiaSem'] = covid_BR_df['Timestamp'].dt.weekday

covid_UF_df['Timestamp'] = pd.to_datetime(covid_UF_df['date'], format='%Y/%m/%d')
covid_UF_df['Ano'] = covid_UF_df['Timestamp'].dt.year
covid_UF_df['Mês'] = covid_UF_df['Timestamp'].dt.month
covid_UF_df['Dia'] = covid_UF_df['Timestamp'].dt.day
covid_UF_df['DiaSem'] = covid_UF_df['Timestamp'].dt.weekday

In [ ]:
covid_BR_df.head(5)

In [ ]:
covid_UF_df.head(5)

In [ ]:
BRpop_df.head(5)

In [ ]:
covid_BR_df['daily_deaths']=covid_BR_df['deaths'].diff(1)
covid_UF_df['daily_deaths']=covid_UF_df['deaths'].diff(1)


In [ ]:
ACL_C_df=ACL_df[ACL_df['Timestamp']>='2020-03-25']
ACR_C_df=ACR_df[ACR_df['Timestamp']>='2020-03-25']

covid_BR_df=covid_BR_df[covid_BR_df['Timestamp']>='2020-03-25']
covid_UF_df=covid_UF_df[covid_UF_df['Timestamp']>='2020-03-25']


In [ ]:
covid_time=covid_BR_df.groupby(['Timestamp'])['daily_deaths'].mean()
ACL_time=ACL_C_df.groupby(['Timestamp'])['Consumo'].mean()
ACR_time=ACR_C_df.groupby(['Timestamp'])['Consumo'].mean()
covid_UF_df = covid_UF_df.sort_values(by = ['state', 'Timestamp'], ascending = [True, True])


In [ ]:
covid_UF_df['daily_deaths']=covid_UF_df['deaths'].diff(1)


In [ ]:
covid_UF_df['daily_deaths'].loc[covid_UF_df['daily_deaths'] < 0] = 0;

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots(figsize=(18, 6))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Covid Deaths', color=color)
ax1.plot(covid_time, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Consumption (MWm)', color=color)  # we already handled the x-label with ax1
#ax2.plot(df0_ts, color=color)
ax2.plot(df1_ts, color=color)
ax2.tick_params(axis='y', labelcolor=color)

color = 'tab:grey'
ax2.set_ylabel('Consumption (MWm)', color=color)  # we already handled the x-label with ax1
ax2.plot(df0_ts, color=color)
#ax2.plot(df1_ts, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

#df0_ts=df0.groupby(['Timestamp'])['Consumo'].sum()/1000
#df1_ts=df1.groupby(['Timestamp'])['Consumo'].sum()/1000
#df_ts=df.groupby(['Timestamp'])['Consumo'].sum()/1000

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots(figsize=(18, 6))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Covid Deaths', color=color)
ax1.plot(covid_time, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Consumption (MWm)', color=color)  # we already handled the x-label with ax1
#ax2.plot(df0_ts, color=color)
ax2.plot(df1_ts, color=color)
ax2.tick_params(axis='y', labelcolor=color)


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

#df0_ts=df0.groupby(['Timestamp'])['Consumo'].sum()/1000
#df1_ts=df1.groupby(['Timestamp'])['Consumo'].sum()/1000
#df_ts=df.groupby(['Timestamp'])['Consumo'].sum()/1000

In [ ]:
df1_ts

In [ ]:
# Create some mock data
fig, ax1 = plt.subplots(figsize=(18, 6))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Covid Deaths', color=color)
ax1.plot(covid_time, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Consumption (MWm)', color=color)  # we already handled the x-label with ax1
ax2.plot(df0_ts, color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Consumption (MWm)', color=color)  # we already handled the x-label with ax1
ax2.plot(df1_ts, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
pivot_df = covid_UF_df.pivot_table(values='daily_deaths',index='state', columns='Mês')
plt.figure(figsize=(20,4))
plt.title('Distribuição de mortes diárias', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Mortes diárias')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');


plt.figure(figsize=(18,6))
sns.barplot(data=pivot_df)

plt.figure(figsize=(18,6))
plt.plot(pivot_df);

In [ ]:
pivot_df = covid_UF_df.pivot_table(values='daily_deaths',index='state', columns='DiaSem')
plt.figure(figsize=(20,4))
plt.title('Distribuição de mortes diárias', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Mortes diárias')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');

plt.figure(figsize=(18,6))
sns.barplot(data=pivot_df)

plt.figure(figsize=(18,6))
plt.plot(pivot_df);

In [ ]:
pivot_df = covid_UF_df.pivot_table(values='daily_deaths', index='DiaSem')
plt.figure(figsize=(10,1))
plt.title('Distribuição de mortes diárias', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Mortes diárias')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g',cmap='Oranges');

pivot_df = covid_UF_df.pivot_table(values='daily_deaths', index='Mês')
plt.figure(figsize=(10,1))
plt.title('Distribuição de mortes diárias', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Mortes diárias')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');

pivot_df = covid_UF_df.pivot_table(values='daily_deaths', index='state')
plt.figure(figsize=(10,1))
plt.title('Distribuição de mortes diárias', fontsize=20);
#plt.xlabel("Ramo de atividade")
plt.ylabel('Mortes diárias')
sns.heatmap(pivot_df.transpose().astype(int), annot=True, fmt='g', cmap='Oranges');

In [ ]:
covid_UF_df.head()

In [ ]:
a=covid_UF_df
a=a.drop(['date','region','cases','Ano','Mês','Dia','DiaSem'], axis=1)

In [ ]:
a.head()

In [ ]:
pC_df = a.pivot_table(values='daily_deaths',index='state')
pC_df = pC_df.drop(index='RR')

pE_0_df = Covid_0_ACL.pivot_table(values='Consumo',index='UF')
pE_1_df = Covid_1_ACL.pivot_table(values='Consumo',index='UF')


#Covid_0_ACL
#Covid_1_ACL

In [ ]:
xxx = pC_df.values.flatten()
yyy = pE_0_df.values.flatten()
zzz = pE_1_df.values.flatten()

kkk=pd.Series(xxx.transpose())
lll=pd.Series(yyy.transpose())
mmm=pd.Series(zzz.transpose())


In [ ]:
BRpop_df.head(5)

In [ ]:
pop=BRpop_df['Pop'].astype(float)
UF=BRpop_df['UF']

In [ ]:
pop

In [ ]:
UF

In [ ]:
#xxx = pC_df.values.flatten()
#yyy = pE_df.values.flatten()

#kkk=pd.Series(xxx.transpose())
#lll=pd.Series(yyy.transpose())

#days=ACL_df['UF'].unique()
#days

Scatter_covid_cons = pd.concat([UF,kkk,lll,mmm,pop], axis=1, keys=['UF','Deaths','Cons_Pre_covid','Cons_Pos_covid','Pop'])


#scatterClass['UF'] = days

In [ ]:
Scatter_covid_cons

In [ ]:
a=Scatter_covid_cons

In [ ]:
a['Deaths']

In [ ]:
a['ratio']=a['Cons_Pos_covid']/a['Cons_Pre_covid']
a['cons_drop']=(1-a['Cons_Pos_covid']/a['Cons_Pre_covid'])*100
a['deaths/Mhab']=(a['Deaths']/a['Pop'])*1000000
a['ratio_norm']=a['ratio']/(a['ratio'].max())
a['deaths_norm']=a['Deaths']/(a['Deaths'].max())
a['cons/Mhab']=(a['Cons_Pre_covid']/a['Pop'])*1000000

In [ ]:
a.head()

In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(x='deaths/Mhab',y='UF', data=Scatter_covid_cons, color='grey', s=100, alpha=0.5)
#sns.scatterplot(x='Deaths',y='Cons_Pos_covid', data=Scatter_covid_cons, palette='twilight_shifted_r', s=100)
#sns.scatterplot(x='Deaths',y='Deaths', data=Scatter_covid_cons, palette='twilight_shifted_r', s=10)
sns.scatterplot(x='cons/Mhab',y='UF', data=Scatter_covid_cons, color='blue', s=100, alpha=0.5)


plt.legend(loc='best', fontsize='medium')
plt.grid(b=None, which='major', axis='both')
plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#plt.text(10, 80,"WEF@Davos", color="gray")



In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(x='cons/Mhab',y='UF', data=Scatter_covid_cons, color='grey', s=100, alpha=0.5)
#sns.scatterplot(x='Deaths',y='Cons_Pos_covid', data=Scatter_covid_cons, palette='twilight_shifted_r', s=100)
#sns.scatterplot(x='Deaths',y='Deaths', data=Scatter_covid_cons, palette='twilight_shifted_r', s=10)

plt.legend(loc='best', fontsize='medium')
plt.grid(b=None, which='major', axis='both')
plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#plt.text(10, 80,"WEF@Davos", color="gray")

In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(x='Deaths',y='cons_drop', data=Scatter_covid_cons, hue='UF', s=100, alpha=0.5)
#sns.scatterplot(x='Deaths',y='Cons_Pos_covid', data=Scatter_covid_cons, palette='twilight_shifted_r', s=100)
#sns.scatterplot(x='Deaths',y='Deaths', data=Scatter_covid_cons, palette='twilight_shifted_r', s=10)

plt.legend(loc='best', fontsize='medium')
plt.grid(b=None, which='major', axis='both')
plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#plt.text(10, 80,"WEF@Davos", color="gray")

plt.ylim((0,30))
plt.xlim((0,30))


In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(x='deaths_norm',y='ratio_norm', data=Scatter_covid_cons, color='grey', s=100, alpha=0.5)
#sns.scatterplot(x='Deaths',y='Cons_Pos_covid', data=Scatter_covid_cons, palette='twilight_shifted_r', s=100)
#sns.scatterplot(x='Deaths',y='Deaths', data=Scatter_covid_cons, palette='twilight_shifted_r', s=10)

plt.legend(loc='best', fontsize='medium')
plt.grid(b=None, which='major', axis='both')
plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#plt.text(10, 80,"WEF@Davos", color="gray")

x=np.array([0,160])
m=1
plt.plot(x, m*x, color='black')

plt.ylabel('Consumo (MWm)');

plt.ylim((0,2))
plt.xlim((0,2))



In [ ]:
plt.figure(figsize=(16,8))
sns.scatterplot(x='Deaths',y='Cons_Pre_covid', data=Scatter_covid_cons, color='grey', s=100, alpha=0.5)
sns.scatterplot(x='Deaths',y='Cons_Pos_covid', data=Scatter_covid_cons, palette='twilight_shifted_r', s=100)
#sns.scatterplot(x='Deaths',y='Deaths', data=Scatter_covid_cons, palette='twilight_shifted_r', s=10)

plt.legend(loc='best', fontsize='medium')
plt.grid(b=None, which='major', axis='both')
plt.title('Scatter - Consumo vs. Queda de consumo - Antes e depois Covid-19', fontsize=20);
#plt.text(10, 80,"WEF@Davos", color="gray")

x=np.array([1,160])
m=1
plt.plot(x, m*x, color='black')


plt.ylim((0,50))
plt.xlim((0,50))